<p style="float:right">
<img src="nasa.png" style="display:inline" />
</p>

# Variable Subsetting with OPeNDAP

For this demonstration, we will examine the [_MEaSUREs Northern Hemisphere Terrestrial Snow Cover Extent Daily 25km EASE-Grid 2.0_](http://nsidc.org/data/docs/measures/nsidc-0530/) dataset from the [NASA MEaSUREs project](https://earthdata.nasa.gov/community/community-data-system-programs/measures-projects). This dataset is managed by [NSIDC DAAC](http://nsidc.org/data/measures/) and made available via, among other methods, the [OPeNDAP](https://en.wikipedia.org/wiki/OPeNDAP) protocol.

### Basic data access with OPeNDAP

The Python `netCDF4` package provides an OPeNDAP client, and we use it here to access snow-cover data from January 1, 2012:

In [ ]:
import netCDF4

url0 = (
    'http://n5eil01u.ecs.nsidc.org:80/opendap/'
    'MEASURES/NSIDC-0530.001/2012.01.01/nhtsd25e2_20120101_v01r01.nc'
)

dataset0 = netCDF4.Dataset(url0)

Among many other pieces of metadata, we can inspect the dataset's title to ensure we have what we want:

In [ ]:
dataset0.title

We can examine the dataset's `variables` attribute to see the names of the attributes containing the actual snow-cover and associated spatio-temporal data:

In [ ]:
for variable in dataset0.variables:
    print(variable)

### Subsetting data by variable with OPeNDAP

A valuable feature of OPeNDAP is its ability to subset data prior to download, to avoid unnecessary storage and transfer costs.

The [OPeNDAP Server Dataset Access Form](http://n5eil01u.ecs.nsidc.org/opendap/MEASURES/NSIDC-0530.001/2012.01.01/nhtsd25e2_20120101_v01r01.nc.html) gives some subsetting guidance. As selections are made in the form, the displayed _Data URL_ is updated to reflect the necessary changes to the OPeNDAP query.

Let's restrict our query to the three variables _Latitude_, _Longitude_ and _Merged Snow Cover Extent_. On the form, when we tick the associated checkboxes **latitude**, **longitude**, and **merged_snow_cover_extent**, the _Data URL_ field is updated based on these selections.

To subset by variable, we simply update our OPeNDAP query with the new _Data URL_:

In [ ]:
url1 = (
    'http://n5eil01u.ecs.nsidc.org:80/opendap/'
    'MEASURES/NSIDC-0530.001/2012.01.01/nhtsd25e2_20120101_v01r01.nc?'
    'latitude[0:1:719][0:1:719],'
    'longitude[0:1:719][0:1:719],'
    'merged_snow_cover_extent[0:1:0][0:1:719][0:1:719]'
)

dataset1 = netCDF4.Dataset(url1)

for variable in dataset1.variables:
    print(variable)

This mechanism could be abstracted in a function parametrized to, for example, obtain a time series variable subset.

Note that the bounds expressions (e.g. `[0:1:719]`) shown in the URL are defaults that request the variable arrays in their entirety. We can verify that the array shapes agree with these expressions:

In [ ]:
for v in ['latitude', 'longitude', 'merged_snow_cover_extent']: 
    print(dataset1.variables[v].shape)

Since we are requesting complete arrays, the bounds expressions could just as well have been omitted. The notebook _Geographical Subsetting with OPeNDAP_ demonstrates how these bounds expressions can be used to limit data to a certain geographical area.

### Displaying data

Let's do a quick, geolocated plot of our snow-cover data.

Our dataset contains helpful documentation on its variables, and we can examine `merged_snow_cover_extent` to see what values this variable takes on, and how to intepret them:

In [ ]:
dataset1.variables['merged_snow_cover_extent']

So, values the the range 10 to 16, inclusive, represent snow cover, and 30 represents ice-covered land.

Let's plot an Arctic basemap and, on top of it, cyan points where our dataset indicates that snow was present, and blue for ice.

In [ ]:
%matplotlib inline

from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np

def snow_ice_plot(snow_ice_map, lat, lon, condition, color, size=1):
    row, col = np.where(condition)
    x, y = snow_ice_map(lon[row, col], lat[row, col])
    snow_ice_map.scatter(x, y, s=size, color=color)

lat0 = dataset1.variables['latitude'][:, :]
lon0 = dataset1.variables['longitude'][:, :]
msce_jan = dataset1.variables['merged_snow_cover_extent'][0, :, :]

plt.figure(figsize=(10, 10))
snow_ice_jan = Basemap(projection='npstere', boundinglat=45, lon_0=300)
snow_ice_jan.drawcoastlines()

snow_ice_plot(snow_ice_jan, lat0, lon0, (msce_jan >= 10) & (msce_jan <= 16), 'Cyan')
snow_ice_plot(snow_ice_jan, lat0, lon0, msce_jan == 30, 'Blue')

### Using a data-access class

As a simplification, we can make use of a prebuilt Pyhton class, `NSIDC0530`, to access remote data. The class constructor takes year, date and month arguments, as well as an optional list of desired variables. (Note that the `latitude` and `longitude` variables are provided with the class and need not be requested.)

Here, we request the `merged_snow_cover_extent` data for June 1, 2012, then plot it with code otherwise nearly identical to that used in the previous example:

In [ ]:
from nsidc0530.interface import NSIDC0530

june_data = NSIDC0530(2012, 6, 1, ['merged_snow_cover_extent'])

lat1 = june_data.latitude
lon1 = june_data.longitude
msce_jun = june_data.merged_snow_cover_extent

plt.figure(figsize=(10, 10))
snow_ice_jun = Basemap(projection='npstere', boundinglat=45, lon_0=300)
snow_ice_jun.drawcoastlines()

snow_ice_plot(snow_ice_jun, lat1, lon1, (msce_jun >= 10) & (msce_jun <= 16), 'Cyan')
snow_ice_plot(snow_ice_jun, lat1, lon1, msce_jun == 30, 'Blue')